In [1]:

num_tiles_row = 8
num_tiles_col = 8
tile_width = 60
tile_height = tile_width
border_width = 1
goals = [(0, 0), (5, 5), (4, 0), (6, 3), (7, 7)]
obstacles = [(2, 3), (3, 3), (1,3), (1,4), (1,5), (3, 2), (5, 3), (4,3),  (3, 1), (6, 6), (6,5), (6, 7), (5, 7)]
font_size = 20
font_offset_x = 30
font_offset_y = 20
goal_counter = 0


class Tile:
    def __init__(self, x, y, color = (255, 255, 255)):
        self.pos = (x, y)
        self.f = 1000
        self.g = 0
        self.h = 0

        self.x = x
        self.y = y

        self.type = 'normal'
        self.parent = None
        self.goal_number = None


    def __repr__(self):
        return str(self.pos)

    def __str__(self):
        return self.type

        


In [2]:
tiles = []

def initialize_tiles():
    for i in range(num_tiles_col):
        temp = []
        for j in range(num_tiles_row):
            tile = Tile(j, i)
            temp.append(tile)
    
        tiles.append(temp)
    
    
    for x, y in obstacles:
        tiles[y][x].type = 'obstacle'
    
    goal = 0
    for e in goals:
        y, x = e
        tiles[x][y].goal_number = goal
        goal+=1
        tiles[x][y].type = 'goal'
initialize_tiles() 

In [3]:
import queue
def dist(x, y):
    return abs(x.x - y.x) + abs(x.y - y.y)

def add(a, b):
    return (a[0] + b[0], a[1] + b[1]) 

def addMove(pos, a):
    if a == 'w':
        return add(pos, (-1, 0))
    if a == 's':
        return add(pos, (1, 0))
    if a == 'a':
        return add(pos, (0, -1))
    if a == 'd':
        return add(pos, (0, 1))
    else:
        return pos

def get_min_index(open_tiles):
    index = 0
    for i in range(len(open_tiles)):
        if open_tiles[i].f < open_tiles[index].f:
            index = i

    return index

def end_path(start, end, path):
    pos = start.pos
    for e in path:
        pos = addMove(pos, e)

    return pos

def isValid(pos, move, prevMov):
    if move == 'w' and prevMov == 's':
        return False
    if move == 's' and prevMov == 'w':
        return False
    if move == 'a' and prevMov == 'd':
        return False
    if move == 'd' and prevMov == 'a':
        return False

    x, y = addMove(pos, move)
    if x == -1 or x == num_tiles_row or y == -1 or y == num_tiles_col:
        return False

    return tiles[x][y].type != 'obstacle'
    

    



def astar(start, end):
    nums = queue.Queue()
    nums.put("")
    path = ""
    prevMove = None
    pos = start.pos
    while pos != end.pos:
        path = nums.get()

        for move in ['w', 'a', 's', 'd']:
            if len(path) > 0:
                prevMove = path[-1]
            temp = path + move
            pos = end_path(start, end, path)
            if isValid(pos, end, path):
                nums.put(temp)

    return path
        

In [4]:
def print_tiles():
    for row in tiles:
        temp = ""
        for e in row:
            temp += str(e) + " "

        print(temp)


In [5]:
astar(tiles[0][0], tiles[4][3])

'ddddsss'

In [6]:
initialize_tiles()

In [7]:
import pygame


pygame.init()
screen = pygame.display.set_mode((800, 600))
running = True

white_square = pygame.Surface([tile_width - 2*border_width, tile_height - 2*border_width])
border_square = pygame.Surface([tile_width, tile_height])
agent_square = pygame.Surface([tile_width - 2*border_width, tile_height - 2*border_width])
obstacle_square = pygame.Surface([tile_width - 2*border_width, tile_height - 2*border_width])
goal_square = pygame.Surface([tile_width - 2*border_width, tile_height - 2*border_width])

white_square.fill((255, 255, 255))
border_square.fill((0, 0, 0))
agent_square.fill((0, 0, 255))
obstacle_square.fill((0, 0, 0))
goal_square.fill((0, 255, 0))

goal_index = 2
y, x = goals[0]
y1, x1 = goals[1]
path = astar(tiles[x][y], tiles[x1][y1])
pos = (x, y)

print(path)


font = pygame.font.SysFont("Ariel", font_size)
while running:
    screen.fill((255, 255, 255))


    for row in tiles:
        for e in row:
            screen.blit(border_square, (e.x*tile_width, e.y*tile_height))
            if e.type == 'normal':
                screen.blit(white_square, (e.x*tile_width + border_width, e.y*tile_height +border_width))
        
            # if e.type=='goal':
            #     #img = font.render(str(e.goal_number), True, (0, 0, 0))

    

    for y, x in obstacles:
        e = tiles[x][y]
        screen.blit(obstacle_square, (e.x*tile_width + border_width, e.y*tile_height +border_width))

    for y, x in goals:
        img = font.render(str(tiles[x][y].goal_number), True, (0, 0, 0))
        screen.blit(img, (x*tile_width + border_width, y*tile_height +border_width))


    x, y = pos
    e = tiles[x][y]
    screen.blit(agent_square, (e.x*tile_width + border_width, e.y*tile_height +border_width))
    
    pygame.display.update()
    pygame.time.wait(1000)

    if len(path) > 0:
        pos = addMove(pos, path[0])
        path = path[1:]
    elif goal_index < len(goals):
        y, x = pos
        y1, x1 = goals[goal_index]
        path = astar(tiles[x][y], tiles[x1][y1])
        print(path, pos, (x1, y1))
        goal_index+=1
        

    

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            pygame.quit()


        
    



pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html
ssssssddwddd
aaasaaww (5, 5) (0, 4)
ssddd (4, 0) (3, 6)
wwddddsss (6, 3) (7, 7)
